In [6]:
import numpy as np
import gmsh
import pandas as pd
from collections import deque, defaultdict
import scipy.io
from scipy.spatial import ConvexHull
from mpl_toolkits.mplot3d import Axes3D  # needed for 3D plotting
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import os

In [2]:
def connectivityExtractor(name):
    file_path = 'Networks/Network_Vessels_' + name +'.mat'
    matlab_data = scipy.io.loadmat(file_path)
    # Extract the 'connectivity' field from the 'Data' structured array
    data_structure = matlab_data['Data']
    connectivity_raw = data_structure['connectivity'][0, 0]  # Access the data (adjust indexing if needed)
    # Reshape or ensure it's a proper 2D array (if required)
    connectivity_data = connectivity_raw.squeeze()
    # Create a DataFrame from the connectivity data
    connectivity_df = pd.DataFrame(connectivity_data, columns=['Parent', 'Daughter1', 'Daughter2', 'Daughter3'])
    connectivity_df.replace(0, np.nan, inplace=True) #ensure all nonexistent vessels have NaN
    connectivity_df.at[0,'Parent']=0 #make sure first vessel is 0 (purposefully removed in last step for ease)
    # Save the DataFrame to inspect it
    return connectivity_df

def nodesExtractor(name): #extracts nodes and their corresponding information
    file_path = 'Networks/Network_Vessels_' + name +'.mat'
    matlab_data = scipy.io.loadmat(file_path)
    # Extract the 'connectivity' field from the 'Data' structured array
    data_structure = matlab_data['nodesC2']
    # Reshape or ensure it's a proper 2D array (if required)
    nodes_data = data_structure.squeeze()
    # Create a DataFrame from the connectivity data
    nodes_df = pd.DataFrame(nodes_data, columns=['NodeID', 'X', 'Y', 'Z', 'Degree'])
    # Save the DataFrame to inspect it
    return nodes_df

def edgesExtractor(name): #extracts segments to create a dataframe of from and to nodes
    file_path = 'Networks/Network_Vessels_' + name +'.mat'
    matlab_data = scipy.io.loadmat(file_path)
    # Extract the 'segments' field
    data_structure = matlab_data['segments']
    # Reshape or ensure it's a proper 2D array (if required)
    edges_data = data_structure.squeeze()
    # Create a DataFrame from the connectivity data
    edge_df = pd.DataFrame(edges_data, columns=['ID', 'From', 'To'])
    # Save the DataFrame to inspect it
    return edge_df
    
def findInputVessel(segments,fromnode,to):
    vessel = segments[((segments['From'] == fromnode)&(segments['To']==to))|((segments['From'] == to)&(segments['To']==fromnode))]
    return int(vessel['ID'])

def mapIDExtractor(name):
    file_path = 'Networks/Network_Vessels_' + name +'.mat'
    matlab_data = scipy.io.loadmat(file_path)
    # Extract the 'mapID' field from the 'Data' structured array
    data_structure = matlab_data['Data']
    map_raw = data_structure['mapIDs'][0, 0]  # Access the data (adjust indexing if needed)
    # Reshape or ensure it's a proper 2D array (if required)
    map_data = map_raw.squeeze()
    # Create a DataFrame from the connectivity data
    map_df = pd.DataFrame(map_data, columns=['New', 'Old'])
    # Save the DataFrame to inspect it
    return map_df

def lobeExtractor(name, vesID):
    data = connectivityExtractor(name)
    
    tree = defaultdict(list)
    for _,row in data.iterrows():
        parent = row['Parent']
        for daughter_col in ['Daughter1','Daughter2','Daughter3']:
            daughter = row[daughter_col]
            if pd.notna(daughter):
                tree[parent].append(daughter)

    visited = set()
    queue = deque([vesID])

    while queue:
        current = queue.popleft()
        if current not in visited:
            visited.add(current)
            queue.extend(tree.get(current,[]))
    
    downstream_df = data[data['Parent'].isin(visited)]
    return downstream_df

def term_nodes_loc(name,lobe_nodes):
    nodes = nodesExtractor(name)
    lobe = nodes[nodes['NodeID'].isin(lobe_nodes)]
    termNodes = lobe[(lobe['Degree'] == 1)]
    return termNodes[['X','Y','Z']]

def node_loc(name,lobe_nodes):
    nodes = nodesExtractor(name)
    lobe = nodes[nodes['NodeID'].isin(lobe_nodes)]
    return lobe[['X','Y','Z']]

def lobeTermLoc(name,fromnode,tonode):
    segments = edgesExtractor(name)
    maps = mapIDExtractor(name)
    vesID = findInputVessel(segments,fromnode,tonode)
    newID = int(maps[maps['Old']==vesID]['New'])
    lobe_ves = lobeExtractor(name,newID)
    new_lobe_ves_ID = lobe_ves['Parent'].to_numpy()
    oldID = maps[maps['New'].isin(new_lobe_ves_ID)]['Old'].to_numpy()
    fromnodes = segments[segments['ID'].isin(oldID)]['From'].to_numpy()
    tonodes = segments[segments['ID'].isin(oldID)]['To'].to_numpy()
    lobe_nodes = np.unique(np.concatenate((fromnodes,tonodes))).astype(int)
    lobe_node_loc = node_loc(name,lobe_nodes)/1000
    #term_nodes = term_nodes_loc(name,lobe_nodes)
    return lobe_node_loc

def compute_mesh_volume():
    types, elem_tags, elem_nodes = gmsh.model.mesh.getElements(dim=3)
    total_volume = 0.0

    for etype, tags, nodes in zip(types,elem_tags,elem_nodes):
        if etype !=4:
            continue
        nodes = np.array(nodes).reshape(-1,4)
        node_tags,node_coords,_ = gmsh.model.mesh.getNodes()
        node_coords = np.array(node_coords).reshape(-1,3)
        node_map = dict(zip(node_tags,node_coords))

        for tet in nodes:
            p0 = node_map[tet[0]]
            p1 = node_map[tet[1]]
            p2 = node_map[tet[2]]
            p3 = node_map[tet[3]]
        
        vol = np.abs(np.dot((p1-p0),np.cross((p2-p0),(p3-p0))))/6
        total_volume += vol
        print(f"Total mesh volume: {total_volume}")
        return total_volume

In [3]:
name = 'm3p4_060407'
left_lobe = lobeTermLoc(name,939,955).to_numpy()

In [ ]:
name = 'm3p4_060407'
left_lobe = lobeTermLoc(name,939,955).to_numpy()
sup1loc = lobeTermLoc(name,991,989).to_numpy()
middleloc = lobeTermLoc(name,1033,1027).to_numpy()
sup2loc = lobeTermLoc(name,1056,1035).to_numpy()
suploc = np.concatenate((sup1loc,sup2loc))
inferiorloc = lobeTermLoc(name,1056,963).to_numpy()
post_cavalloc = lobeTermLoc(name,1056,1087).to_numpy()

In [4]:
left_lobe

array([[ 9.72002511, 10.85026059, 10.75984175],
       [10.03649105, 11.16672653, 10.6242135 ],
       [12.07091491, 13.92450109,  7.0978788 ],
       [12.25175258, 13.87929167,  7.36913532],
       [11.21193594, 11.61882072, 15.05473657],
       [11.21193594, 12.07091491, 14.46701412],
       [13.29156922, 14.01491993, 11.3475642 ],
       [13.15594097, 13.83408225, 11.5736113 ],
       [12.07091491,  7.86643893, 16.7726945 ],
       [11.39277362,  7.45955416, 16.13976263],
       [12.97510329, 11.39277362, 13.06552213],
       [12.88468445, 11.21193594, 12.83947503],
       [13.4724069 , 15.8232967 , 12.47779968],
       [13.15594097, 11.5736113 , 10.03649105],
       [12.61342794, 12.07091491, 10.44337582],
       [ 9.76523453, 11.25714536, 13.24635981],
       [10.21732872, 11.3475642 , 13.51761632],
       [13.38198806, 14.51222354, 12.02570549],
       [11.5736113 , 11.70923955, 17.85772056],
       [11.3475642 , 10.75984175, 17.04395101],
       [10.57900408, 10.21732872,  9.900

In [ ]:
name = 'm1p4_053007'
left_lobe = lobeTermLoc(name, 1810, 1811).to_numpy()
points = np.unique(left_lobe, axis=0)

# Build a convex hull
hull = ConvexHull(points)
volume = hull.volume
# Initialize Gmsh
gmsh.initialize()
gmsh.model.add("volume_from_convex_hull")

# Create discrete surface entity
gmsh.model.addDiscreteEntity(2, 1)  # dim=2, tag=1

# Add nodes from your point cloud
node_tags = np.arange(1, len(points) + 1)
gmsh.model.mesh.addNodes(2, 1, node_tags.tolist(), points.flatten().tolist())

# Add triangular faces from the convex hull
triangles = hull.simplices + 1  # Gmsh uses 1-based indexing
gmsh.model.mesh.addElements(2, 1, [2], [np.arange(1, len(triangles) + 1)], [triangles.flatten()])

# Convert mesh surface into geometry
gmsh.model.mesh.classifySurfaces(30 * np.pi / 180., True, False, 180 * np.pi / 180.)
gmsh.model.mesh.createGeometry()
gmsh.model.geo.synchronize()

# Build volume from surface shell
surfaces = gmsh.model.getEntities(2)
surface_tags = [s[1] for s in surfaces]
loop = gmsh.model.geo.addSurfaceLoop(surface_tags)
volume = gmsh.model.geo.addVolume([loop])
gmsh.model.geo.synchronize()
gmsh.write("./Mesh/" + name + "_left_lobe.stl")
# Tag volume for export
gmsh.model.addPhysicalGroup(3, [volume], name="EnclosedVolume")

# Generate 3D tetrahedral mesh
gmsh.option.setNumber('Mesh.CharacteristicLengthMin',.1) #min mesh distance
gmsh.option.setNumber('Mesh.CharacteristicLengthMax',.5)#max mesh distance
#ensures mesh doesn't overoptimize and get stuck in a loop
gmsh.option.setNumber('Mesh.MeshOnlyVisible',1)
gmsh.option.setNumber('Mesh.HighOrderOptimize',0)
gmsh.option.setNumber('Mesh.RecombineAll',0)
gmsh.option.setNumber('Mesh.Smoothing',0)
gmsh.option.setNumber('Mesh.Optimize',0)
gmsh.model.mesh.generate(3)
#visualizes mesh before writing to unv
gmsh.fltk.run()
#volume = compute_mesh_volume()
# Export as UNV
gmsh.write("./Mesh/" + name + "_left_lobe.unv")

gmsh.finalize()
print(volume)

/var/folders/js/z_h_pjy55jl6y6y98vrg66cc0000gs/T/ipykernel_14777/3927154232.py:42: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/js/z_h_pjy55jl6y6y98vrg66cc0000gs/T/ipykernel_14777/3927154232.py:95: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead



Info    : Classifying surfaces (angle: 30)...
Info    : Found 89 model surfaces
Info    : Found 150 model curves
Info    : Done classifying surfaces (Wall 0.00158279s, CPU 0.001725s)
Info    : Creating geometry of discrete curves...
Info    : Done creating geometry of discrete curves (Wall 4.4458e-05s, CPU 5e-05s)
Info    : Creating geometry of discrete surfaces...
Info    : [ 10%] Discrete surface 6 is planar, simplifying parametrization                                        
Info    : [ 10%] Discrete surface 9 is planar, simplifying parametrization
Info    : [ 10%] Discrete surface 10 is planar, simplifying parametrization
Info    : [ 20%] Discrete surface 11 is planar, simplifying parametrization                                       
Info    : [ 20%] Discrete surface 12 is planar, simplifying parametrization
Info    : [ 20%] Discrete surface 13 is planar, simplifying parametrization
Info    : [ 20%] Discrete surface 14 is planar, simplifying parametrization
Info    : [ 20%] Discre

: 

In [ ]:
def plot_sphere(ax, center, radius=0.1, color='r'):
    """Plot a sphere centered at `center` with given radius and color on the Axes3D `ax`."""
    u = np.linspace(0, 2 * np.pi, 12)
    v = np.linspace(0, np.pi, 12)
    x = center[0] + radius * np.outer(np.cos(u), np.sin(v))
    y = center[1] + radius * np.outer(np.sin(u), np.sin(v))
    z = center[2] + radius * np.outer(np.ones(np.size(u)), np.cos(v))
    ax.plot_surface(x, y, z, color=color, alpha=0.6, linewidth=0)

name = 'm3p4_060407'
left_lobe = lobeTermLoc(name,851,926).to_numpy()
sup1loc = lobeTermLoc(name,885,894).to_numpy()
sup2loc = lobeTermLoc(name,884,864).to_numpy()
suploc = np.concatenate((sup1loc,sup2loc))
middle1loc = lobeTermLoc(name,865,61).to_numpy()
middle2loc = lobeTermLoc(name,878,880).to_numpy()
middleloc = np.concatenate((middle1loc,middle2loc))
inferiorloc = lobeTermLoc(name,878,879).to_numpy()
post_cavalloc = lobeTermLoc(name,865,920).to_numpy()

fig = go.Figure()

# Add each set as a scatter3d trace
fig.add_trace(go.Scatter3d(
    x=left_lobe[:,0], y=left_lobe[:,1], z=left_lobe[:,2],
    mode='markers',
    marker=dict(size=8, color='red', symbol='circle'),
    name='left_lobe'
))

fig.add_trace(go.Scatter3d(
    x=suploc[:,0], y=suploc[:,1], z=suploc[:,2],
    mode='markers',
    marker=dict(size=8, color='green', symbol='diamond'),
    name='suploc'
))

fig.add_trace(go.Scatter3d(
    x=middleloc[:,0], y=middleloc[:,1], z=middleloc[:,2],
    mode='markers',
    marker=dict(size=8, color='blue', symbol='square'),
    name='middleloc'
))

fig.add_trace(go.Scatter3d(
    x=inferiorloc[:,0], y=inferiorloc[:,1], z=inferiorloc[:,2],
    mode='markers',
    marker=dict(size=8, color='magenta', symbol='cross'),
    name='inferiorloc'
))

fig.add_trace(go.Scatter3d(
    x=post_cavalloc[:,0], y=post_cavalloc[:,1], z=post_cavalloc[:,2],
    mode='markers',
    marker=dict(size=8, color='cyan', symbol='cross'),
    name='post_cavalloc'
))

fig.update_layout(scene=dict(
    xaxis_title='X',
    yaxis_title='Y',
    zaxis_title='Z'
),
    width=800,
    height=700,
    title='3D Scatter Plot of Points'
)

# To save the plot as an interactive HTML file
fig.write_html('3d_points_plot.html')

# To display in a Jupyter notebook or compatible environment
fig.show()